# Adwind RAT / JRAT
Detects javaw.exe in AppData folder as used by Adwind / JRAT

## Rule Content
```
- action: global
  title: Adwind RAT / JRAT
  id: 1fac1481-2dbc-48b2-9096-753c49b4ec71
  status: experimental
  description: Detects javaw.exe in AppData folder as used by Adwind / JRAT
  references:
  - https://www.hybrid-analysis.com/sample/ba86fa0d4b6af2db0656a88b1dd29f36fe362473ae8ad04255c4e52f214a541c?environmentId=100
  - https://www.first.org/resources/papers/conf2017/Advanced-Incident-Detection-and-Threat-Hunting-using-Sysmon-and-Splunk.pdf
  author: Florian Roth, Tom Ueltschi
  date: 2017/11/10
  modified: 2018/12/11
  tags:
  - attack.execution
  - attack.t1064
  detection:
    condition: selection
  level: high
- logsource:
    category: process_creation
    product: windows
  detection:
    selection:
      CommandLine:
      - '*\AppData\Roaming\Oracle*\java*.exe *'
      - '*cscript.exe *Retrive*.vbs *'
- logsource:
    product: windows
    service: sysmon
  detection:
    selection:
      EventID: 11
      TargetFilename:
      - '*\AppData\Roaming\Oracle\bin\java*.exe'
      - '*\Retrive*.vbs'
- logsource:
    product: windows
    service: sysmon
  detection:
    selection:
      EventID: 13
      TargetObject: \REGISTRY\MACHINE\SOFTWARE\Microsoft\Windows\CurrentVersion\Run*
      Details: '%AppData%\Roaming\Oracle\bin\\*'

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\\AppData\\Roaming\\Oracle*\\java*.exe\ * OR *cscript.exe\ *Retrive*.vbs\ *)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"11" AND file_name.keyword:(*\\AppData\\Roaming\\Oracle\\bin\\java*.exe OR *\\Retrive*.vbs))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:\\REGISTRY\\MACHINE\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run* AND registry_key_value.keyword:%AppData%\\Roaming\\Oracle\\bin\\*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()